In [ ]:
import pandas as pd
import boto3
import sagemaker


In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()


In [ ]:
data_dir = 'plagdata'
prefix = 'plagiarism_detector_ourmodel'
input_data = sagemaker_session.upload_data(
    path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)


In [ ]:
empchk = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empchk.append(obj.key)
    print(obj.key)
assert len(empchk) != 0, 'S3 bucket is empty.'
print('Test passed!')


In [ ]:
from sagemaker.sklearn.estimator import SKLearn
output_path = 's3://{}/{}'.format(bucket, prefix)
estimator = SKLearn(entry_point='train.py',
                    framework_version="0.20.0",
                    py_version="py3",
                    source_dir='srcsklearn',
                    role=role,
                    instance_count=1,
                    instance_type='ml.c4.xlarge',
                    sagemaker_session=sagemaker_session,
                    output_path=output_path,
                    )


In [ ]:
%%time
estimator.fit({'train': input_data})


In [ ]:
%%time
predictor = estimator.deploy(
    initial_instance_count=1, instance_type='ml.t2.medium')


In [ ]:
import os
datatst= pd.read_csv(os.path.join(
    data_dir, "test.csv"), header=None, names=None)
ytst = datatst.iloc[:, 0]
xtst = datatst.iloc[:, 1:]

In [ ]:
testypdct = predictor.predict(xtst)
assert len(testypdct) == len(ytssrcsklearnd number of predictions'
print('Test cases are passed!')


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
nametarget = ['notplg', 'P']
accuracy = accuracy_score(ytst, testypdct)
print('Accuracy got:', accuracy)
print(classification_report(ytst.values, testypdct, target_names=nametarget))
print('\nPredicted class labels are: ')
print(testypdct)
print('\nTrue class labels are: ')
print(ytst.values)


In [ ]:
predictor.delete_endpoint()


In [ ]:
delbucket = boto3.resource('s3').Bucket(bucket)
delbucket.objects.all().delete()


In [ ]:
def evaluate(predictor, test_features, test_labels, verbose=True):
    test_preds = np.squeeze(np.rint(predictor.predict(xtst))).astype(int)
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=[
              'actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall of ourmodel got:', recall))
        print("{:<11} {:.3f}".format('Precision of ourmodel got:', precision))
        print("{:<11} {:.3f}".format('Accuracy of ourmodel got:', accuracy))
        print()
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn,
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


In [ ]:
import os
datatst= pd.read_csv(os.path.join(
    data_dir, "test.csv"), header=None, names=None)
ytst = datatst.iloc[:, 0]
xtst = datatst.iloc[:, 1:]


In [ ]:
testypdct = predictor.predict(xtst)
assert len(testypdct) == len(ytst), 'Unexpected number of predictions got'
print('Test cases are succesfully passed!')


In [ ]:
import numpy as np
testypdct = np.squeeze(np.rint(predictor.predict(xtst))).astype(int)
testypdct


In [ ]:

print('\nPredicted class labels are: ')
print(testypdct)
print('\nTrue class labels are: ')
print(ytst.values)


In [ ]:
metrics = evaluate(predictor, xtst, ytst.values, True)


In [ ]:
def enddel(predictor):
    try:
        boto3.client('sagemaker').enddel(
            EndpointName=predictor.endpoint)
        print('Deleted {}'.format(predictor.endpoint))
    except:
        print('Already deleted: {}'.format(predictor.endpoint))


In [ ]:
enddel(predictor)
